# Start H2O Cluster

_**Note**: The `os.system` command below is used solely for the H2O Aquarium training platform._

In [ ]:
import os
os.system('/home/h2o/bin/startup')
!sleep 10

Start by importing `h2o` and creating a connection to the server. The parameters used in `h2o.init` will depend on your specific environment. 

In [ ]:
import h2o
h2o.init(url='http://localhost:54321/h2o')

Some commentary needed below

In [ ]:
h2o.cluster().show_status()

In [ ]:
h2o.cluster().show_status(True)

In [ ]:
df = h2o.import_file("/home/h2o/data/prostate/prostate.csv")

In [ ]:
df.summary()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sd()

In [ ]:
%matplotlib inline

import pylab as pl
df.as_data_frame().hist()
pl.show()

In [ ]:
df.type

In [ ]:
df.shape

In [ ]:
df.col_names

In [ ]:
print(df['CAPSULE'].unique())
print(df['DPROS'].levels)

## Generalized Liner Model : Linear Regression

In [ ]:
y = 'CAPSULE'
x = df.col_names
x.remove(y)

In [ ]:
print("Response = " + y)
print("Predictors = " + str(x))

In [ ]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
glm = H2OGeneralizedLinearEstimator()

In [ ]:
glm.train(x = x, y = y, training_frame = df)

In [ ]:
print(glm)

In [ ]:
glm.model_performance()

In [ ]:
print(glm.rmse())
print(glm.mse())
print(glm.r2())

In [ ]:
glm.coef()

In [ ]:
train, valid = df.split_frame(ratios=[.9])

In [ ]:
print(df.shape)
print(train.shape)
print(valid.shape)

In [ ]:
train, valid, test = df.split_frame(ratios=[.8, .1])

In [ ]:
print(df.shape)
print(train.shape)
print(valid.shape)
print(test.shape)

In [ ]:
glm.train(x= x, y = y, training_frame=train, validation_frame=valid)

In [ ]:
glm

In [ ]:
print(glm.model_performance(test_data=test).rmse())
print(glm.model_performance(test_data=valid).rmse())
print(glm.model_performance(test_data=train).rmse())

In [ ]:
print(glm.model_performance(test_data=test).r2())
print(glm.model_performance(test_data=valid).r2())
print(glm.model_performance(test_data=train).r2())

In [ ]:
glm.coef()

In [ ]:
glm.predict(test_data=test)

## Generalized Linear Model : Logistic Regression

In [ ]:
df['CAPSULE'].summary()

In [ ]:
df['CAPSULE'] = df['CAPSULE'].asfactor()

In [ ]:
df['CAPSULE'].summary()

In [ ]:
df['CAPSULE'].levels()

In [ ]:
train, valid, test = df.split_frame(ratios=[.8,.1])

In [ ]:
glm_logistic = H2OGeneralizedLinearEstimator(family = "binomial")

In [ ]:
glm_logistic.train(x=x, y= y, training_frame=train, validation_frame=valid, 
                   model_id="glm_logistic"
                  )


In [ ]:
glm_logistic

In [ ]:
glm_logistic.varimp()

In [ ]:
glm_logistic.confusion_matrix()

In [ ]:
glm_logistic.predict(test_data=test)

## Gradient Boosting Model

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
gbm = H2OGradientBoostingEstimator()

In [ ]:
gbm.train(x=x, y =y, training_frame=train, validation_frame=valid)

In [ ]:
gbm

In [ ]:
gbm.confusion_matrix()

In [ ]:
gbm.varimp()

In [ ]:
gbm.varimp_plot()

## Machine learning with cross validation

In [ ]:
gbm_cv3 = H2OGradientBoostingEstimator(nfolds=3)

In [ ]:
gbm_cv3.train(x=x, y=y, training_frame=train)

In [ ]:
gbm_cv3

In [ ]:
gbm_cv3.cross_validation_predictions()

In [ ]:
gbm_cv4 = H2OGradientBoostingEstimator(keep_cross_validation_predictions = True, nfolds=4)

In [ ]:
gbm_cv4.train(x=x, y=y, training_frame=train)

In [ ]:
gbm_cv4.cross_validation_fold_assignment

## Random Forest

In [ ]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [ ]:
drf = H2ORandomForestEstimator(nfolds=5, keep_cross_validation_predictions=True)

In [ ]:
drf.train(x=x, y = y, training_frame=train)

In [ ]:
drf

In [ ]:
drf.confusion_matrix()

In [ ]:
drf.gains_lift()

## Deep Learning

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [ ]:
##random_fold = random.randint(3,10)
dl_model = H2ODeepLearningEstimator(nfolds=5,fold_assignment="Modulo",hidden=[20,20],epochs=10)

In [ ]:
dl_model.train(x=x, y=y, training_frame=train)

In [ ]:
dl_model

In [ ]:
train.hist

## AutoML

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(max_runtime_secs=30)

In [ ]:
aml.train(x = x, y = y, training_frame = train)

In [ ]:
aml.leaderboard

In [ ]:
aml.leader

In [ ]:
aml.leader.auc()

In [ ]:
aml.leader.confusion_matrix()

In [ ]:
aml.leader.predict(test_data=test)

In [ ]:
aml.leader.model_performance(test_data=test)

In [ ]:
h2o.cluster().shutdown()